# Get All Custom CIS Policies

In [ ]:
import sys
import json
import datetime
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

In [ ]:
def getAllCISPolicies():
    policies = []
    page, totalPages, totalResources = 0, 99, 0
    while page <= totalPages:
        getAssetSearchURL = HOSTNAME + "api/3/policies?size=500&sort=ASC&scannedOnly=true&page=" + str(page)

        print("Hostname : " + getAssetSearchURL)
        r = requests.get(getAssetSearchURL, headers=HEADER, verify=False)
        if r.status_code == 200:
            r = r.json()
            policies += [res["title"] for res in r["resources"] if "CIS" in res["title"] or "SCB" in res["title"]]
            if page == 0:
                totalPages, totalResources = r["page"]["totalPages"], r["page"]["totalResources"]
        else:
            print("[Error] Request Connection Error.")
            sys.exit()
        page += 1

    print("Total " + str(len(policies)) + " of " + str(totalResources))
    return sorted(policies)

In [ ]:
getPolicies = getAllCISPolicies()
getPolicies = list(set(getPolicies))

In [ ]:
len(getPolicies), getPolicies

In [ ]:
nowTime = datetime.datetime.now().strftime("(%y%m%d-%H%M%S) ")
filepath="outputs/" + nowTime + "allCISPolicies" + ".json"

In [ ]:
with open(filepath, 'w', encoding="utf8") as f:
    f.write(json.dumps(getPolicies, indent=4))
print("\n[Success Exporting] " + filepath)

## Update Policy Config